In [1]:
import requests
import time
import os
from typing import Generator

import jwt

from data_types import TextMsg, ImageMsg, TextMsgList, MsgList, CharacterMeta


# 智谱开放平台API key，参考 https://open.bigmodel.cn/usercenter/apikeys
API_KEY: str = os.getenv("API_KEY", "")

In [24]:
class ApiKeyNotSet(ValueError):
    pass


def verify_api_key_not_empty():
    if not API_KEY:
        raise ApiKeyNotSet


def generate_token(apikey: str, exp_seconds: int) -> str:
    # reference: https://open.bigmodel.cn/dev/api#nosdk
    try:
        id, secret = apikey.split(".")
    except Exception as e:
        raise Exception("invalid apikey", e)
 
    payload = {
        "api_key": id,
        "exp": int(round(time.time() * 1000)) + exp_seconds * 1000,
        "timestamp": int(round(time.time() * 1000)),
    }
 
    return jwt.encode(
        payload,
        secret,
        algorithm="HS256",
        headers={"alg": "HS256", "sign_type": "SIGN"},
    )


def get_characterglm_response(messages: TextMsgList, meta: CharacterMeta) -> Generator[str, None, None]:
    """ 通过http调用characterglm """
    # Reference: https://open.bigmodel.cn/dev/api#characterglm
    verify_api_key_not_empty()
    url = "https://open.bigmodel.cn/api/paas/v3/model-api/charglm-3/sse-invoke"
    resp = requests.post(
        url,
        headers={"Authorization": generate_token(API_KEY, 1800)},
        json=dict(
            model="charglm-3",
            meta=meta,
            prompt=messages,
            incremental=True)
    )
    resp.raise_for_status()
    
    # 解析响应（非官方实现）
    sep = b':'
    last_event = None
    for line in resp.iter_lines():
        if not line or line.startswith(sep):
            continue
        field, value = line.split(sep, maxsplit=1)
        if field == b'event':
            last_event = value
        elif field == b'data' and last_event == b'add':
            yield value.decode()


def get_characterglm_response_via_sdk(messages: TextMsgList, meta: CharacterMeta) -> Generator[str, None, None]:
    """ 通过旧版sdk调用characterglm """
    # 与get_characterglm_response等价
    # Reference: https://open.bigmodel.cn/dev/api#characterglm
    # 需要安装旧版sdk，zhipuai==1.0.7
    import zhipuai
    verify_api_key_not_empty()
    zhipuai.api_key = API_KEY
    response = zhipuai.model_api.sse_invoke(
        model="charglm-3",
        meta= meta,
        prompt= messages,
        incremental=True
    )
    for event in response.events():
        if event.event == 'add':
            yield event.data


def get_chatglm_response_via_sdk(messages: TextMsgList) -> Generator[str, None, None]:
    """ 通过sdk调用chatglm """
    # reference: https://open.bigmodel.cn/dev/api#glm-3-turbo  `GLM-3-Turbo`相关内容
    # 需要安装新版zhipuai
    from zhipuai import ZhipuAI
    verify_api_key_not_empty()
    client = ZhipuAI(api_key=API_KEY) # 请填写您自己的APIKey
    response = client.chat.completions.create(
        model="glm-4",  # 填写需要调用的模型名称
        messages=messages,
        stream=True,
    )
    for chunk in response:
        yield chunk.choices[0].delta.content

In [25]:
def generate_role_appearance(role_profile: str) -> Generator[str, None, None]:
    """ 用chatglm生成角色的外貌描写 """
    
    instruction = f"""
请从下列文本中，抽取人物的外貌描写。若文本中不包含外貌描写，请你推测人物的性别、年龄，并生成一段外貌描写。要求：
1. 只生成外貌描写，不要生成任何多余的内容。
2. 外貌描写不能包含敏感词，人物形象需得体。
3. 尽量用短语描写，而不是完整的句子。
4. 不要超过50字

文本：
{role_profile}
"""
    return get_chatglm_response_via_sdk(
        messages=[
            {
                "role": "user",
                "content": instruction.strip()
            }
        ]
    )

In [32]:
def generate_role_info(user_text: str) -> Generator[str, None, None]:
    """ 用chatglm生成角色的人设描写 """
    
    instruction = f"""
请从下列文本中，抽取2位人物的人设描写。若文本中的信息，请你推测人物的性别、年龄，并生成人物角色人设。要求：
1. 角色人设信息应包含身份、外貌、背景、性格、年龄等。
2. 外貌描写不能包含敏感词，人物形象需得体。
3. 尽量用短语描写，而不是完整的句子。
4. 返回的结果格式如下： {{
    "bot1_name": "角色名1",
    "bot2_name": "角色名2", 
    "bot1_info": "角色人设1",
    "bot2_info": "用户人设2"}}

文本：
{user_text}
"""
    return get_chatglm_response_via_sdk(
        messages=[
            {
                "role": "user",
                "content": instruction.strip()
            }
        ]
    )

In [14]:
def gen_role(user_text: str) -> Generator[str, None, None]:
    """ 用chatglm生成角色的人设描写 并序列为dict对象"""
    response = generate_role_info(user_text)
    import json
    role_obj = json.loads("".join(response))
    return role_obj

In [5]:
c = """
剑胆琴心：琴剑江湖】

角色一：柳七弦

身份：江湖侠女，琴师

年龄：二十三岁

外貌：长发如瀑，肤白胜雪，眉目如画，一袭青衣，腰悬古琴。

性格：聪慧机敏，冷静沉着，心怀仁义，对朋友极尽忠诚，对敌人则毫不留情。

背景：柳七弦自幼父母双亡，被一位琴师收养，并传授琴艺。她天资聪颖，十六岁便琴艺有成，开始行走江湖。她以琴会友，以剑护身，行侠仗义，逐渐在江湖中闯出了“琴心剑胆”的名号。

武器：古琴“秋水”，剑“碧落”

技能：琴音武功，剑法高超

角色二：风无痕

身份：江湖侠客，剑客

年龄：二十五岁

外貌：短发利落，剑眉星目，身形矫健，一袭白衣，腰悬长剑。

性格：热血仗义，直率豪爽，有些冲动，但内心善良，对朋友极尽忠诚。

背景：风无痕出身江湖世家，自幼习武，剑法高超。他十八岁开始行走江湖，以剑会友，以武行侠，行侠仗义，逐渐在江湖中闯出了“无痕剑”的名号。

武器：剑“无痕”

技能：剑法高超，轻功了得

故事梗概：

柳七弦与风无痕在一次江湖风波中相识，两人志趣相投，结为好友。他们一起闯荡江湖，以琴剑结合，行侠仗义，助人解难。在一次对抗邪恶势力的战斗中，两人联手，大败敌人，保护了江湖的和平。他们的故事在江湖中流传，成为一段佳话。
"""


In [33]:
 response = generate_role_info(c)

In [34]:
response

<generator object get_chatglm_response_via_sdk at 0x7f7e0295b940>

In [35]:
image_prompt = "".join(response)

In [36]:
image_prompt.strip()

'{\n    "bot1_name": "柳七弦",\n    "bot2_name": "风无痕",\n    "bot1_info": "身份：江湖侠女，琴师；年龄：二十三岁；外貌：长发青衣，肤白眉画；性格：聪慧冷静，仁义忠诚；背景：孤儿，琴师养女，琴剑双修。",\n    "bot2_info": "身份：江湖侠客，剑客；年龄：二十五岁；外貌：短发白衣，剑眉星目；性格：热血豪爽，冲动善良；背景：江湖世家，剑法高，轻功了得。"\n}'

In [37]:
print(image_prompt)

{
    "bot1_name": "柳七弦",
    "bot2_name": "风无痕",
    "bot1_info": "身份：江湖侠女，琴师；年龄：二十三岁；外貌：长发青衣，肤白眉画；性格：聪慧冷静，仁义忠诚；背景：孤儿，琴师养女，琴剑双修。",
    "bot2_info": "身份：江湖侠客，剑客；年龄：二十五岁；外貌：短发白衣，剑眉星目；性格：热血豪爽，冲动善良；背景：江湖世家，剑法高，轻功了得。"
}


In [38]:
a  = gen_role(c)

In [39]:
a

{'bot1_name': '柳七弦',
 'bot2_name': '风无痕',
 'bot1_info': '身份：江湖侠女，琴师；外貌：长发青衣，肤白眉画；性格：聪慧冷静，仁义忠诚；年龄：二十三岁；背景：孤儿，琴师养女，琴剑双修，江湖成名；武器：古琴‘秋水’，剑‘碧落’；技能：琴音武功，剑法高超',
 'bot2_info': '身份：江湖侠客，剑客；外貌：短发白衣，剑眉星目；性格：热血豪爽，冲动善良；年龄：二十五岁；背景：江湖世家，剑法了得，以武会友，江湖成名；武器：剑‘无痕’；技能：剑法高超，轻功出众'}